In [1]:
from typing import List, Optional

import os
import random
import logging
from collections import OrderedDict
import h5py
import torch
import numpy as np
from tqdm import tqdm
from transformers import BertTokenizer, CharacterBertTokenizer

from utils.text import convert_to_unicode, truncate_seq_pair

WORKDIR = os.environ['WORKDIR']

In [2]:
class PreTrainingExample:
    r"""
    A single pre-training example: sentence pair with targets for NSP and MLM.
    Args:
    tokens (:obj:`list(str)`):
        Input tokens: [CLS] A A A [SEP] B B B [SEP]
    segment_ids (:obj:`list(int)`):
        Segment ids corresponding to the input tokens: 0 0 0 0 0 1 1 1 1
    mlm_positions (:obj:`list(int)`):
        Indices of masked (altered) tokens.
    mlm_labels (:obj:`list(str)`):
        Original tokens for masked (altered) positions.
    is_random_next (:obj:`bool`):
        Whether text span B is random (NSP target is False when this is True).
    """

    def __init__(self,
        tokens: List[str],
        segment_ids: List[int],
        mlm_positions: List[int],
        mlm_labels: List[str],
        is_random_next: bool
    ):
        self.tokens = tokens
        self.segment_ids = segment_ids
        self.is_random_next = is_random_next
        self.mlm_positions = mlm_positions
        self.mlm_labels = mlm_labels

    def __str__(self):
        string = ""
        string += "tokens: %s\n" % \
            (" ".join([convert_to_unicode(x) for x in self.tokens]))
        string += "segment_ids: %s\n" % \
            (" ".join([str(x) for x in self.segment_ids]))
        string += "is_random_next: %s\n" % \
            self.is_random_next
        string += "mlm_positions: %s\n" % \
            (" ".join([str(x) for x in self.mlm_positions]))
        string += "mlm_labels: %s\n" % \
            (" ".join([convert_to_unicode(x) for x in self.mlm_labels]))
        string += "\n"
        return string

    def __repr__(self):
        return self.__str__()

In [3]:
class PreTrainingDataGenerator:
    r"""
    Generates MLM/NSP examples for BERT/CharacterBERT from a single shard.
    Args:
    shard_fpath (:obj:`str`):
        Path to a corpus shard contraining one sentence per line and a blank
        line between sentences from different documents.
    output_directory (:obj:`str`)
        'Path to a directory for saving the output .hdf5 file.'
    is_character_bert (:obj:`bool`):
        Whether to create pre-training for CharacterBERT of BERT. When this is
        True, data is generated for CharacterBERT.
    duplication_factor (:obj:`int`):
        How many iterations over the shard contents. The higher this value the
        more we randomly generate examples from the same sentences/documents.
    max_input_length (:obj:`int`):
        Maximum sequence length for the model input. Usually this is set to 512.
    short_input_probability (:obj:`float`):
        Probability of generating an exemple with a shorter input. When this
        happens, a random number between 2 and `max_input_length` is picked and
        a short input of that size is generated.
    max_masked_tokens_per_input (:obj:`int`):
        Hard limit on the number of tokens that can be masked.
    masked_tokens_ratio (:obj:`float`):
        Proportion of input tokens that we attempt to mask. If this is higher than
        `max_masked_tokens_per_input` then we only mask the allowed maximum number.
    random_seed (:obj:`float`):
        A random seed to set before randomly generating pre-training examples.
    verbose (:obj:`bool`, `optional`, defaults to :obj:`True`):
        Whether to print logs and progress bars. It is useful to turn this off
        if running multiple processes (too many logs)
    """
    def __init__(self,
        shard_fpath: str,
        output_directory: str,
        is_character_bert: bool,
        duplication_factor: int,
        max_input_length: int,
        short_input_probability: float,
        max_masked_tokens_per_input: int,
        masked_tokens_ratio: float,
        random_seed: int,
        verbose: Optional[bool] = True,
    ):
        self.verbose = verbose
        if not self.verbose:
            logging.disable(logging.WARNING)

        self.random_seed = random_seed
        self.set_all_random_seeds()

        self.shard_fpath = shard_fpath
        self.output_directory = output_directory
        self.is_character_bert = is_character_bert
        self.duplication_factor = duplication_factor
        self.max_input_length = max_input_length
        self.short_input_probability = short_input_probability
        self.max_masked_tokens_per_input = max_masked_tokens_per_input
        self.masked_tokens_ratio = masked_tokens_ratio

        self.pretraining_examples = []

        # Load tokenizer / masked language modeling vocabulary:
        if self.is_character_bert:
            # Here, MLM vocabulary is a list of most frequent tokens in the corpus
            logging.info('Using a "word" vocabulary for MLM.')
            self.tokenizer = CharacterBertTokenizer()
            corpus_name = os.path.basename(os.path.dirname(self.shard_fpath))
            self.mlm_vocabulary = [
                line.strip().split()[-1]
                for line in open(
                    os.path.join(
                        WORKDIR, 'data', 'mlm_vocabularies',
                        corpus_name, 'mlm_vocab.txt'),
                    'r', encoding='utf-8')
            ]
        else:
            # Here, MLM vocabulary is the same as the wordpiece vocabulary
            logging.info('Using a WordPiece vocabulary for MLM.')
            self.tokenizer = BertTokenizer.from_pretrained(
                os.path.join(WORKDIR, 'data', 'bert-base-uncased'))
            self.mlm_vocabulary = list(self.tokenizer.vocab)
        logging.info('Example tokens: %s', random.sample(self.mlm_vocabulary, 10))


    def set_all_random_seeds(self):
        r"""Sets all random seeds to `self.random_seed`"""
        random.seed(self.random_seed)
        np.random.seed(self.random_seed)
        torch.manual_seed(self.random_seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(self.random_seed)
        logging.info("Random seed set to: %d", self.random_seed)


    def generate_pretraining_examples(self):
        r"""Generates pre-training examples (MLM/NSP) from the input shard."""
        logging.info('BEGIN: Generate Pre-training Examples')

        # Gather token sequences by document
        documents = self.read_documents_from_shard()

        # We run through the corpus as many times as `self.duplication_factor`
        iterator = range(self.duplication_factor)
        if self.verbose:
            iterator = tqdm(iterator, desc='Iterations over all documents')
        for _ in iterator:
            for doc_id in range(len(documents)):
                new_pretraining_examples = \
                    self.generate_examples_from_document(
                        document_id=doc_id,
                        all_documents=documents
                    )
                self.pretraining_examples.extend(new_pretraining_examples)

        # Shuffle the examples within the shard's pre-training data
        logging.info("Shuffling examples...")
        random.shuffle(self.pretraining_examples)

        logging.info('END: Generate Pre-training Examples')


    def read_documents_from_shard(self):
        r"""
        Groups token sequences by document for the input shard then returns
        a list of documents where each document is a set of token sequences.
        """
        # Input file format:
        # (1) One sentence per line. These should ideally be actual sentences, not
        # entire paragraphs or arbitrary spans of text. (Because we use the
        # sentence boundaries for the "next sentence prediction" task).
        # (2) Blank lines between documents. Document boundaries are needed so
        # that the "next sentence prediction" task doesn't span between documents.
        counter = 0
        documents = [[]]
        logging.info("Grouping sentences by document from: %s", self.shard_fpath)
        with open(self.shard_fpath, "r", encoding='utf-8') as f:
            for line in f:
                line = convert_to_unicode(line)
                line = line.strip()
                if line == '':
                    documents.append([])  # New document
                else:
                    # BERT: basic tokenization + wordpiece tokenization
                    # CharacterBERT: basic tokenization only
                    tokens = self.tokenizer.tokenize(line)
                    if tokens:
                        # Add tokens to last document
                        documents[-1].append(tokens)
                        if counter < 2:
                            logging.info("Example of a sentence: %s", tokens)
                    counter += 1

        logging.info("Removing empty documents (if any)...")
        # There shouldn't be any empty documents, but just to be extra safe
        documents = [x for x in documents if x]

        logging.info("Shuffling documents...")
        random.shuffle(documents)
        return documents


    def generate_examples_from_document(self, document_id, all_documents):
        """Returns a number of `PreTrainingExample` objects from a single document."""
        document = all_documents[document_id]

        # Account for [CLS], [SEP], [SEP]
        max_num_tokens = self.max_input_length - 3

        # We *usually* want to fill up the entire input since we are padding
        # to `max_input_length` anyways, so short input sequences are generally wasted
        # computation. However, we *sometimes*
        # (i.e., short_input_probability = 0.1 = 10% of the time) want to use shorter
        # input sequences to minimize the mismatch between pre-training and fine-tuning.
        # The `target_input_length` is just a rough target however, whereas `max_input_length`
        # is a hard limit.
        target_input_length = max_num_tokens
        if random.random() < self.short_input_probability:
            target_input_length = random.randint(2, max_num_tokens)

        # We DON'T just concatenate all of the tokens from a document into a long
        # sequence and choose an arbitrary split point because this would make the
        # next sentence prediction task too easy. Instead, we split the input into
        # segments "A" and "B" based on the actual "sentences" provided by the user input.
        examples = []
        current_chunk = []
        current_length = 0
        i = 0
        while i < len(document):
            sentence = document[i]
            current_chunk.append(sentence)
            current_length += len(sentence)

            no_more_sentences_in_document = (i == (len(document) - 1))
            exceeded_target_length = (current_length >= target_input_length)
            if no_more_sentences_in_document or exceeded_target_length:
                if current_chunk:

                    # Building the segment `A`:

                    tokens_a = []
                    # `a_end` is how many sentences from `current_chunk` go into `A`.
                    if len(current_chunk) > 1:
                        a_end = random.randint(1, len(current_chunk) - 1)
                    else:
                        a_end = 1
                    for j in range(a_end):
                        tokens_a.extend(current_chunk[j])

                    # Building the segment `B`:
                    tokens_b = []
                    is_random_next = False  # By default, `B` follows `A`

                    # In 50% of the cases,
                    # or if the chunk only has one sentence,
                    # segment `B` is from another random document
                    # (does not follow `A`, i.e. NSP target is False)
                    if len(current_chunk) == 1 or random.random() < 0.5:
                        is_random_next = True
                        target_b_length = target_input_length - len(tokens_a)

                        # Since we didn't actually use these sentences, we
                        # "put them back" so they don't go to waste.
                        num_unused_sentences = len(current_chunk) - a_end
                        i -= num_unused_sentences

                        # This should rarely go for more than one iteration for large
                        # corpora. However, just to be safe, we try to make sure that
                        # the random document is not the same as the document
                        # we're processing.
                        # We don't use a while loop to avoid looping indefinitely
                        # if there is only one document (shouldn't happen)
                        for _ in range(10):
                            random_document_id = random.randint(0, len(all_documents) - 1)
                            if random_document_id != document_id:
                                break

                        # Add tokens using sentences from a random document
                        random_document = all_documents[random_document_id]
                        random_start = random.randint(0, len(random_document) - 1)
                        for j in range(random_start, len(random_document)):
                            tokens_b.extend(random_document[j])
                            if len(tokens_b) >= target_b_length:
                                break

                    # In all other cases, use the actual next sentences
                    # (j >= a_end) to build segment `B`
                    else:
                        is_random_next = False
                        for j in range(a_end, len(current_chunk)):
                            tokens_b.extend(current_chunk[j])

                    # Truncate sentence pair to the target length
                    truncate_seq_pair(tokens_a, tokens_b, target_input_length)
                    assert (len(tokens_a) >= 1) and (len(tokens_b) >= 1)

                    # Building the actual input for the model:
                    # 1 - Adding CLS/SEP to [A, B]
                    # 2 - Building segment ids
                    # 3 - Masking tokens
                    tokens = []
                    segment_ids = []

                    tokens.append("[CLS]")
                    segment_ids.append(0)

                    for token in tokens_a:
                        tokens.append(token)
                        segment_ids.append(0)

                    tokens.append("[SEP]")
                    segment_ids.append(0)

                    for token in tokens_b:
                        tokens.append(token)
                        segment_ids.append(1)

                    tokens.append("[SEP]")
                    segment_ids.append(1)

                    (transformed_tokens, mlm_positions, mlm_labels) = \
                        self.generate_mlm_instances_from_tokens(
                            input_tokens=tokens)

                    examples.append(
                        PreTrainingExample(
                            tokens=transformed_tokens,  # Original tokens with random tokens changed
                            segment_ids=segment_ids,  # The input's segment ids
                            is_random_next=is_random_next,  # For Next Sentence Prediction
                            mlm_positions=mlm_positions,  # For Masked Language Modeling
                            mlm_labels=mlm_labels  # For Masked Language Modeling
                        )
                    )
                current_chunk = []
                current_length = 0
            i += 1

        return examples


    def generate_mlm_instances_from_tokens(self, input_tokens):
        """Generates instances for the Masked Language Modelling objective."""
        candidate_tokens = []
        for (i, token) in enumerate(input_tokens):
            if token == "[CLS]" or token == "[SEP]":
                continue
            candidate_tokens.append((i, token))
        random.shuffle(candidate_tokens)

        # Make a copy of the original input.
        # This will be transformed by randomly altering its tokens
        # and will be the actual input for the model
        output_tokens = input_tokens.copy()

        num_to_predict = max(
            1, int(round(len(input_tokens) * self.masked_tokens_ratio)))
        num_to_predict = min(
            num_to_predict, self.max_masked_tokens_per_input)

        # Create MLM instances
        mlm_instances = []
        covered_indices = set()
        for index, token in candidate_tokens:
            if len(mlm_instances) >= num_to_predict:
                break
            if index in covered_indices:
                continue
            if self.is_character_bert:
                # CharacterBERT: only mask tokens that are in the
                # MLM vocabulary (i.e. most frequent tokens in the corpus)
                if token not in self.mlm_vocabulary:
                    continue
            else:
                # BERT: all tokens are in the MLM vocabulary anyway
                # as MLM vocabulary == WordPiece vocabulary and all tokens
                # are WordPieces
                pass
            covered_indices.add(index)

            # Compute the token that will be placed at `index`
            masked_token = None
            # 80% of the time, replace with [MASK]
            if random.random() < 0.8:
                masked_token = "[MASK]"
            else:
                # 10% of the time, keep original
                if random.random() < 0.5:
                    masked_token = token
                # 10% of the time, replace with random word
                else:
                    masked_token = random.choice(self.mlm_vocabulary)

            # Replace token at `index`
            output_tokens[index] = masked_token

            # Add MLM instance
            original_token = input_tokens[index]
            mlm_instances.append((index, original_token))

        #  Sort instances according to the masked tokens indices
        mlm_instances = sorted(mlm_instances, key=lambda x: x[0])

        mlm_positions, mlm_labels = [], []
        for (index, original_token) in mlm_instances:
            mlm_positions.append(index)
            mlm_labels.append(original_token)

        # Returns:
        #     - output_tokens: transformed tokens
        #     - mlm_positions: indices of masked tokens
        #     - mlm_labels: original token for each masked token
        return (output_tokens, mlm_positions, mlm_labels)


    def write_examples_to_hdf5(self):
        """Converts examples to tensors and saves it in hdf5 format."""

        logging.info('BEGIN: Write Pre-Training Examples to hdf5 File')

        total_written = 0
        features = OrderedDict()
        token_to_id = {w: i for i, w in enumerate(self.mlm_vocabulary)}

        num_instances = len(self.pretraining_examples)

        #### Initializing empty tensors ####

        # NOTE: (!important) here we assume that padding index is 0
        # for both BERT's and CharacterBERT's input ids, which is the default.
        if self.is_character_bert:
            features["input_ids"] = np.zeros(
                [
                    num_instances,
                    self.max_input_length,
                    self.tokenizer._mapper.max_word_length
                ],
                dtype="int32"
            )
        else:
            features["input_ids"] = np.zeros(
                [
                    num_instances,
                    self.max_input_length,
                ],
                dtype="int32")
        features["input_mask"] = np.zeros(
            [num_instances, self.max_input_length], dtype="int32")
        features["segment_ids"] = np.zeros(
            [num_instances, self.max_input_length], dtype="int32")
        features["masked_lm_positions"] = np.zeros(
            [num_instances, self.max_masked_tokens_per_input], dtype="int32")
        features["masked_lm_ids"] = np.zeros(
            [num_instances, self.max_masked_tokens_per_input], dtype="int32")
        features["next_sentence_labels"] = np.zeros(
            num_instances, dtype="int32")

        #### Filling the tensors ####

        iterator = self.pretraining_examples
        if self.verbose:
            iterator = tqdm(
                iterator, total=num_instances,
                desc='Converting pre-training examples to tensors'
            )
        for instance_id, instance in enumerate(iterator):
            input_ids = self.tokenizer.convert_tokens_to_ids(instance.tokens)
            segment_ids = list(instance.segment_ids)
            input_mask = [1] * len(segment_ids)
            assert len(input_ids) <= self.max_input_length
            assert len(segment_ids) <= self.max_input_length

            # Padding:
            while len(segment_ids) < self.max_input_length:
                if self.is_character_bert:
                    input_ids.append([0] * self.tokenizer._mapper.max_word_length)
                else:
                    input_ids.append(0)
                input_mask.append(0)
                segment_ids.append(0)

            assert len(input_ids) == self.max_input_length
            assert len(input_mask) == self.max_input_length
            assert len(segment_ids) == self.max_input_length


            mlm_positions = list(instance.mlm_positions)
            if self.is_character_bert:
                masked_lm_ids = [
                    token_to_id[label]
                    for label in instance.mlm_labels]
            else:
                # Since the MLM output layer is a copy of the input Embedding matrix
                masked_lm_ids = \
                    self.tokenizer.convert_tokens_to_ids(
                        instance.mlm_labels)

            # NOTE: I'm not very sure about this part...
            # it seems that we fill up the mlm positions with the CLS position
            while len(mlm_positions) < self.max_masked_tokens_per_input:
                mlm_positions.append(0)
                masked_lm_ids.append(0)

            # NOTE: in the original code `is_random_next` is 1, we change it here
            # so that NSP target is True when not `is_random_next`
            next_sentence_label = 1 if (not instance.is_random_next) else 0

            features["input_ids"][instance_id] = input_ids
            features["input_mask"][instance_id] = input_mask
            features["segment_ids"][instance_id] = segment_ids
            features["masked_lm_positions"][instance_id] = mlm_positions
            features["masked_lm_ids"][instance_id] = masked_lm_ids
            features["next_sentence_labels"][instance_id] = next_sentence_label

            total_written += 1

        # Saving pretraining data as an .hdf5 file
        os.makedirs(self.output_directory, exist_ok=True)
        basename = os.path.basename(self.shard_fpath)
        output_hdf5_fpath = os.path.join(
            self.output_directory, basename.replace('.txt', '.hdf5'))

        logging.info("Saving data...")
        f = h5py.File(output_hdf5_fpath, 'w')
        f.create_dataset(
            "input_ids",
            data=features["input_ids"],
            dtype='i4', compression='gzip'
        )
        f.create_dataset(
            "input_mask",
            data=features["input_mask"],
            dtype='i1', compression='gzip'
        )
        f.create_dataset(
            "segment_ids",
            data=features["segment_ids"],
            dtype='i1', compression='gzip'
        )
        f.create_dataset(
            "masked_lm_positions",
            data=features["masked_lm_positions"],
            dtype='i4', compression='gzip'
        )
        f.create_dataset(
            "masked_lm_ids",
            data=features["masked_lm_ids"],
            dtype='i4', compression='gzip'
        )
        f.create_dataset(
            "next_sentence_labels",
            data=features["next_sentence_labels"],
            dtype='i1', compression='gzip'
        )
        f.flush()
        f.close()

        logging.info('END: Write Pre-Training Examples to hdf5 File')

In [4]:
output_directory = "data/hdf5/80_12/"
# max_input_length = 80
# max_input_length = 20
max_input_length = 120
# max_masked_tokens_per_input = 12
# max_masked_tokens_per_input = 3
max_masked_tokens_per_input = 18
is_character_bert = True
short_input_probability = 0.10
masked_tokens_ratio = 0.15
random_seed = 42
duplication_factor = 1
shard_fpath = 'data\\shrads\\long_text.formatted.txt'

In [5]:
verbose = True
if not verbose:
    logging.disable(logging.WARNING)

random_seed = random_seed

# set_all_random_seeds()
r"""Sets all random seeds to `random_seed`"""
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(random_seed)
logging.info("Random seed set to: %d", random_seed)

pretraining_examples = []

# Load tokenizer / masked language modeling vocabulary:
if is_character_bert:
    # Here, MLM vocabulary is a list of most frequent tokens in the corpus
    logging.info('Using a "word" vocabulary for MLM.')
    tokenizer = CharacterBertTokenizer(strip_accents=None, do_lower_case=None)
    corpus_name = os.path.basename(os.path.dirname(shard_fpath))
    mlm_vocabulary = [
        line.strip().split()[-1]
        for line in open(
            os.path.join(
                WORKDIR, 'data', 'mlm_vocabularies',
                corpus_name, 'mlm_vocab.txt'),
            'r', encoding='utf-8')
    ]
else:
    # Here, MLM vocabulary is the same as the wordpiece vocabulary
    logging.info('Using a WordPiece vocabulary for MLM.')
    tokenizer = BertTokenizer.from_pretrained(
        os.path.join(WORKDIR, 'data', 'bert-base-uncased'))
    mlm_vocabulary = list(tokenizer.vocab)
logging.info('Example tokens: %s', random.sample(mlm_vocabulary, 10))

In [6]:
# Input file format:
# (1) One sentence per line. These should ideally be actual sentences, not
# entire paragraphs or arbitrary spans of text. (Because we use the
# sentence boundaries for the "next sentence prediction" task).
# (2) Blank lines between documents. Document boundaries are needed so
# that the "next sentence prediction" task doesn't span between documents.
counter = 0
documents = [[]]
logging.info("Grouping sentences by document from: %s", shard_fpath)
with open(shard_fpath, "r", encoding='utf-8') as f:
    for line in f:
        line = convert_to_unicode(line)
        line = line.strip()
        if line == '':
            documents.append([])  # New document
        else:
            # BERT: basic tokenization + wordpiece tokenization
            # CharacterBERT: basic tokenization only
            tokens = tokenizer.tokenize(line)
            if tokens:
                # Add tokens to last document
                documents[-1].append(tokens)
                if counter < 2:
                    logging.info("Example of a sentence: %s", tokens)
            counter += 1

logging.info("Removing empty documents (if any)...")
# There shouldn't be any empty documents, but just to be extra safe
documents = [x for x in documents if x]

logging.info("Shuffling documents...")
random.shuffle(documents)

In [7]:
print(len(documents))

1


In [10]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(documents[0], test_size = 0.2, random_state=42)

In [11]:
documents[0] = train
# documents[0] = test

In [12]:
temp = train

In [13]:
len(train)

833930

In [14]:
# train_len = int(0.8 * len(documents[0]))
# documents[0] = documents[0][:train_len]
# documents[0] = documents[0][train_len:]

In [15]:
def generate_examples_from_document(document_id, all_documents):
    """Returns a number of `PreTrainingExample` objects from a single document."""
    document = all_documents[document_id]

    # Account for [CLS], [SEP], [SEP]
    max_num_tokens = max_input_length - 3

    # We *usually* want to fill up the entire input since we are padding
    # to `max_input_length` anyways, so short input sequences are generally wasted
    # computation. However, we *sometimes*
    # (i.e., short_input_probability = 0.1 = 10% of the time) want to use shorter
    # input sequences to minimize the mismatch between pre-training and fine-tuning.
    # The `target_input_length` is just a rough target however, whereas `max_input_length`
    # is a hard limit.
    target_input_length = max_num_tokens

    # We DON'T just concatenate all of the tokens from a document into a long
    # sequence and choose an arbitrary split point because this would make the
    # next sentence prediction task too easy. Instead, we split the input into
    # segments "A" and "B" based on the actual "sentences" provided by the user input.
    examples = []
    current_chunk = []
    current_length = 0
    #train_len = int(0.8 * len(document))
    i = 0
#     i = train_len
#     random.shuffle(document)
    
    while i  < len(document):
#     while i < train_len:
        sentence = document[i]
        current_chunk.append(sentence)
        current_length = len(sentence)

        exceeded_target_length = (current_length >= target_input_length)
        if not exceeded_target_length:
            if current_chunk:
                # Building the segment `A`:

                tokens_a = []
                # `a_end` is how many sentences from `current_chunk` go into `A`.
                if len(current_chunk) > 1:
                    a_end = random.randint(1, len(current_chunk) - 1)
                else:
                    a_end = 1
                for j in range(a_end):
                    tokens_a.extend(current_chunk[j])

                # Truncate sentence pair to the target length
                tokens_b = []
                truncate_seq_pair(tokens_a, tokens_b, target_input_length)
                assert (len(tokens_a) >= 1)

                # Building the actual input for the model:
                # 1 - Adding CLS/SEP to [A, B]
                # 2 - Building segment ids
                # 3 - Masking tokens
                tokens = []
                segment_ids = []

                tokens.append("[CLS]")
                segment_ids.append(0)

                for token in tokens_a:
                    tokens.append(token)
                    segment_ids.append(0)

                tokens.append("[SEP]")
                segment_ids.append(0)

                (transformed_tokens, mlm_positions, mlm_labels) = \
                    generate_mlm_instances_from_tokens(
                        input_tokens=tokens)

                examples.append(
                    PreTrainingExample(
                        tokens=transformed_tokens,  # Original tokens with random tokens changed
                        segment_ids=segment_ids,  # The input's segment ids
                        is_random_next=False,  # For Next Sentence Prediction
                        mlm_positions=mlm_positions,  # For Masked Language Modeling
                        mlm_labels=mlm_labels  # For Masked Language Modeling
                    )
                )
            current_chunk = []
            current_length = 0
        i += 1

    return examples

In [16]:
def generate_mlm_instances_from_tokens(input_tokens):
    """Generates instances for the Masked Language Modelling objective."""
    candidate_tokens = []
    for (i, token) in enumerate(input_tokens):
        if token == "[CLS]" or token == "[SEP]":
            continue
        candidate_tokens.append((i, token))
    random.shuffle(candidate_tokens)

    # Make a copy of the original input.
    # This will be transformed by randomly altering its tokens
    # and will be the actual input for the model
    output_tokens = input_tokens.copy()

    num_to_predict = max(
        1, int(round(len(input_tokens) * masked_tokens_ratio)))
    num_to_predict = min(
        num_to_predict, max_masked_tokens_per_input)

    # Create MLM instances
    mlm_instances = []
    covered_indices = set()
    for index, token in candidate_tokens:
        if len(mlm_instances) >= num_to_predict:
            break
        if index in covered_indices:
            continue
        if is_character_bert:
            # CharacterBERT: only mask tokens that are in the
            # MLM vocabulary (i.e. most frequent tokens in the corpus)
            if token not in mlm_vocabulary:
                continue
        else:
            # BERT: all tokens are in the MLM vocabulary anyway
            # as MLM vocabulary == WordPiece vocabulary and all tokens
            # are WordPieces
            pass
        covered_indices.add(index)

        # Compute the token that will be placed at `index`
        masked_token = None
        # 80% of the time, replace with [MASK]
        if random.random() < 0.8:
            masked_token = "[MASK]"
        else:
            # 10% of the time, keep original
            if random.random() < 0.5:
                masked_token = token
            # 10% of the time, replace with random word
            else:
                masked_token = random.choice(mlm_vocabulary)

        # Replace token at `index`
        output_tokens[index] = masked_token

        # Add MLM instance
        original_token = input_tokens[index]
        mlm_instances.append((index, original_token))

    #  Sort instances according to the masked tokens indices
    mlm_instances = sorted(mlm_instances, key=lambda x: x[0])

    mlm_positions, mlm_labels = [], []
    for (index, original_token) in mlm_instances:
        mlm_positions.append(index)
        mlm_labels.append(original_token)

    # Returns:
    #     - output_tokens: transformed tokens
    #     - mlm_positions: indices of masked tokens
    #     - mlm_labels: original token for each masked token
    return (output_tokens, mlm_positions, mlm_labels)

In [17]:
# temp = [[0]]
# temp[0] = documents[0][:5]
# temp

In [18]:
# generate_examples_from_document(
#                 document_id=0,
#                 all_documents=temp
#             )

In [19]:
pretraining_examples = list()

In [20]:
# We run through the corpus as many times as `duplication_factor`
iterator = range(duplication_factor)
if verbose:
    iterator = tqdm(iterator, desc='Iterations over all documents')
for _ in iterator:
    for doc_id in range(len(documents)):
        new_pretraining_examples = \
            generate_examples_from_document(
                document_id=doc_id,
                all_documents=documents
            )
        pretraining_examples.extend(new_pretraining_examples)
        
logging.info('END: Generate Pre-training Examples')

Iterations over all documents: 100%|████████████████████████████████████████████████████| 1/1 [06:36<00:00, 396.14s/it]


In [21]:
def write_examples_to_hdf5():
    """Converts examples to tensors and saves it in hdf5 format."""

    logging.info('BEGIN: Write Pre-Training Examples to hdf5 File')

    total_written = 0
    features = OrderedDict()
    token_to_id = {w: i for i, w in enumerate(mlm_vocabulary)}

    num_instances = len(pretraining_examples)

    #### Initializing empty tensors ####

    # NOTE: (!important) here we assume that padding index is 0
    # for both BERT's and CharacterBERT's input ids, which is the default.
    if is_character_bert:
        features["input_ids"] = np.zeros(
            [
                num_instances,
                max_input_length,
                tokenizer._mapper.max_word_length
            ],
            dtype="int32"
        )
    else:
        features["input_ids"] = np.zeros(
            [
                num_instances,
                max_input_length,
            ],
            dtype="int32")
    features["input_mask"] = np.zeros(
        [num_instances, max_input_length], dtype="int32")
    features["segment_ids"] = np.zeros(
        [num_instances, max_input_length], dtype="int32")
    features["masked_lm_positions"] = np.zeros(
        [num_instances, max_masked_tokens_per_input], dtype="int32")
    features["masked_lm_ids"] = np.zeros(
        [num_instances, max_masked_tokens_per_input], dtype="int32")
    features["next_sentence_labels"] = np.zeros(
        num_instances, dtype="int32")

    #### Filling the tensors ####

    iterator = pretraining_examples
    if verbose:
        iterator = tqdm(
            iterator, total=num_instances,
            desc='Converting pre-training examples to tensors'
        )
    for instance_id, instance in enumerate(iterator):
        input_ids = tokenizer.convert_tokens_to_ids(instance.tokens)
        segment_ids = list(instance.segment_ids)
        input_mask = [1] * len(segment_ids)
        assert len(input_ids) <= max_input_length
        assert len(segment_ids) <= max_input_length

        # Padding:
        while len(segment_ids) < max_input_length:
            if is_character_bert:
                input_ids.append([0] * tokenizer._mapper.max_word_length)
            else:
                input_ids.append(0)
            input_mask.append(0)
            segment_ids.append(0)

        assert len(input_ids) == max_input_length
        assert len(input_mask) == max_input_length
        assert len(segment_ids) == max_input_length


        mlm_positions = list(instance.mlm_positions)
        if is_character_bert:
            masked_lm_ids = [
                token_to_id[label]
                for label in instance.mlm_labels]
        else:
            # Since the MLM output layer is a copy of the input Embedding matrix
            masked_lm_ids = \
                tokenizer.convert_tokens_to_ids(
                    instance.mlm_labels)

        # NOTE: I'm not very sure about this part...
        # it seems that we fill up the mlm positions with the CLS position
        while len(mlm_positions) < max_masked_tokens_per_input:
            mlm_positions.append(0)
            masked_lm_ids.append(0)

        # NOTE: in the original code `is_random_next` is 1, we change it here
        # so that NSP target is True when not `is_random_next`
        next_sentence_label = 1 if (not instance.is_random_next) else 0

        features["input_ids"][instance_id] = input_ids
        features["input_mask"][instance_id] = input_mask
        features["segment_ids"][instance_id] = segment_ids
        features["masked_lm_positions"][instance_id] = mlm_positions
        features["masked_lm_ids"][instance_id] = masked_lm_ids
#         features["next_sentence_labels"][instance_id] = next_sentence_label

        total_written += 1

    # Saving pretraining data as an .hdf5 file
    os.makedirs(output_directory, exist_ok=True)
    basename = os.path.basename(shard_fpath)
    output_hdf5_fpath = os.path.join(
        output_directory, basename.replace('.txt', '.hdf5'))

    logging.info("Saving data...")
    f = h5py.File(output_hdf5_fpath, 'w')
    f.create_dataset(
        "input_ids",
        data=features["input_ids"],
        dtype='i4', compression='gzip'
    )
    f.create_dataset(
        "input_mask",
        data=features["input_mask"],
        dtype='i1', compression='gzip'
    )
    f.create_dataset(
        "segment_ids",
        data=features["segment_ids"],
        dtype='i1', compression='gzip'
    )
    f.create_dataset(
        "masked_lm_positions",
        data=features["masked_lm_positions"],
        dtype='i4', compression='gzip'
    )
    f.create_dataset(
        "masked_lm_ids",
        data=features["masked_lm_ids"],
        dtype='i4', compression='gzip'
    )
    f.create_dataset(
        "next_sentence_labels",
        data=features["next_sentence_labels"],
        dtype='i1', compression='gzip'
    )
    f.flush()
    f.close()

    logging.info('END: Write Pre-Training Examples to hdf5 File')

In [24]:
# Shuffle the examples within the shard's pre-training data
logging.info("Shuffling examples...")
random.shuffle(pretraining_examples)

write_examples_to_hdf5()

MemoryError: Unable to allocate 382. MiB for an array with shape (833930, 120) and data type int32

In [22]:
len(pretraining_examples)

833930

In [23]:
pretraining_examples

[tokens: [CLS] একটু পর সোহেল সেখান থেকে পালিয়ে যান । এর আধা ঘণ্টা পরেই তিনি মারা যান । পুতুলের ভাই মোজাম্মেল [MASK] অভিযোগের পরিপ্রেক্ষিতে পুলিশ লাশ ময়নাতদন্তের জন্য [MASK] মেডিকেলে পাঠিয়েছে । [MASK] [MASK] রাতেই একটি অপমৃত্যু মামলা হয়েছে । ময়নাতদন্তের গণপিটুনি আসার পর পুতুলের মৃত্যুর কারণ জানা যাবে । এদিকে [MASK] বিএনপি 1111111111 জনের তালিকা কেন্দ্রীয় কার্যালয়ে পাঠিয়েছিল । সংস্করণে [MASK] থেকে কেন্দ্র [MASK] চূড়ান্ত করেছে । নবাগত উল্লাপাড়া নাচের মেয়র পদে মনোনয়নপ্রত্যাশীদের সাক্ষাৎকার [MASK] রোববার অনুষ্ঠিত হবে । [SEP]
 segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 is_random_next: False
 mlm_positions: 20 27 31 32 40 50 59 60 63 67 69 74
 mlm_labels: হকের দিনাজপুর তবে শুক্রবার প্রতিবেদন জেলা তাঁদের মধ্যে দুজনকে তবে পৌরসভার আজ
 ,
 tokens: [CLS] কাল টাইব্রেকারে কোরিয়ার পোচন এফসির মূল্যবান দুটি [MASK] ঠেকিয়েছেন । [SEP]
 segment_ids: 0 0 0 0 0 0 0 0 0 0 0 

In [30]:
for each in pretraining_examples:
    if "র্যাবের" in each.tokens:
        print(each)

tokens: [CLS] এ সময় জামায়াতের কর্মীরা র্যাবের কাছ থেকে [MASK] ছিনিয়ে নেন [MASK] [SEP]
segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0
is_random_next: False
mlm_positions: 8 11
mlm_labels: আসামি ।


tokens: [CLS] সরেজমিনে দেখা যায় র্যাবের তল্লাশি চৌকিটি [MASK] জায়গা ও নন্দীপাড়ার [MASK] সড়কে । [SEP]
segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
is_random_next: False
mlm_positions: 7 11
mlm_labels: শেখের সংযোগ


tokens: [CLS] র্যাবের দাবি তাঁরা সবাই জেএমবির সারোয়ার তামিম [MASK] সক্রিয় সদস্য [MASK] [SEP]
segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0
is_random_next: False
mlm_positions: 8 11
mlm_labels: গ্রুপের ।


tokens: [CLS] আদালত পরিচালনা করেন র্যাবের নির্বাহী ম্যাজিস্ট্রেট ও আইন কর্মকর্তা [MASK] আলম । [SEP]
segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0
is_random_next: False
mlm_positions: 7 10
mlm_labels: ও সারওয়ার


tokens: [CLS] খবর পেয়ে র্যাবের একটি দল সেখানে [MASK] চালায় । [SEP]
segment_ids: 0 0 0 0 0 0 0 0 0 0 0
is_random_next: False
mlm_positions: 4 7
mlm_labels: একটি অভিযান


tokens: [CLS] বিকেলে কারও

tokens: [CLS] এ সময় র্যাবের গাড়ি [MASK] করে গুলি ছোড়া হয় । [SEP]
segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0
is_random_next: False
mlm_positions: 5 8
mlm_labels: লক্ষ্য ছোড়া


tokens: [CLS] এ [MASK] র্যাবের [MASK] গ্রেপ্তার হওয়া রাফিকুলকে এখনো থানায় সোপর্দ করা হয়নি । [SEP]
segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
is_random_next: False
mlm_positions: 2 4
mlm_labels: ছাড়া হাতে


tokens: [CLS] [MASK] র্যাবের নজরদারিতে আছে । [SEP]
segment_ids: 0 0 0 0 0 0 0
is_random_next: False
mlm_positions: 1
mlm_labels: তারাও


tokens: [CLS] [MASK] র্যাবের সদস্যরা তাঁকে ধরে ফেলেন । [SEP]
segment_ids: 0 0 0 0 0 0 0 0 0
is_random_next: False
mlm_positions: 1
mlm_labels: তবে


tokens: [CLS] বিস্তারিত তথ্য [MASK] আজ বেলা 1111111111 র্যাবের ব্রিফিং করার কথা [MASK] [SEP]
segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0
is_random_next: False
mlm_positions: 3 11
mlm_labels: জানাতে ।


tokens: [CLS] তাঁদের [MASK] র্যাবের ছয়জন । [SEP]
segment_ids: 0 0 0 0 0 0 0
is_random_next: False
mlm_positions: 2
mlm_labels: মধ্যে


tokens:

tokens: [CLS] 1111111111 আগস্ট ময়মনসিংহের নান্দাইলে র্যাবের সঙ্গে বন্দুকযুদ্ধে যুক্তরাষ্ট্র নিহত হন [MASK] [SEP]
segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0
is_random_next: False
mlm_positions: 8 11
mlm_labels: শফিউল ।


tokens: [CLS] এতে র্যাবের দুই সদস্য আহত হন টাইমিং [SEP]
segment_ids: 0 0 0 0 0 0 0 0 0
is_random_next: False
mlm_positions: 7
mlm_labels: ।


tokens: [CLS] ঘটনার [MASK] র্যাবের কাছে [MASK] দিয়েছেন তিনি । [SEP]
segment_ids: 0 0 0 0 0 0 0 0 0 0
is_random_next: False
mlm_positions: 2 5
mlm_labels: বিষয়ে স্বীকারোক্তি


tokens: [CLS] র্যাবের ভাষ্যমতে এ ব্যাপারে ইমনের পরিবার র্যাবের ফরিদপুর ক্যাম্পের [MASK] যোগাযোগ করে । [SEP]
segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
is_random_next: False
mlm_positions: 10 13
mlm_labels: সঙ্গে ।


tokens: [CLS] র্যাবের অভিযান টের পেয়ে তাঁদের আরও এলগার সহযোগী পালিয়ে গেছেন । [SEP]
segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0
is_random_next: False
mlm_positions: 4 7
mlm_labels: পেয়ে কিছু


tokens: [CLS] পুলিশ [MASK] র্যাবের পক্ষ থেকে [MASK] হয়েছে তাঁরা 

tokens: [CLS] ইমনের পরিবার র্যাবের সঙ্গে [MASK] করে । [SEP]
segment_ids: 0 0 0 0 0 0 0 0 0
is_random_next: False
mlm_positions: 5
mlm_labels: যোগাযোগ


tokens: [CLS] পুলিশের [MASK] তখন র্যাবের গাড়ির [MASK] নেয় । [SEP]
segment_ids: 0 0 0 0 0 0 0 0 0 0
is_random_next: False
mlm_positions: 2 6
mlm_labels: গাড়িটি পিছু


tokens: [CLS] র্যাব বলেছে [MASK] পর আক্কাসকে প্রথমে র্যাবের ভৈরব ক্যাম্পে নেওয়া [MASK] । [SEP]
segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0
is_random_next: False
mlm_positions: 3 11
mlm_labels: গ্রেপ্তারের হয়


tokens: [CLS] তিন মাসেও চাকরি না হওয়ায় জাহাঙ্গীর বিষয়টি লিখিতভাবে র্যাবের ফরিদপুর ক্যাম্পে জানান [MASK] [SEP]
segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
is_random_next: False
mlm_positions: 1 13
mlm_labels: তিন ।


tokens: [CLS] এ ছাড়া র্যাবের আরও [MASK] সাবেক কর্মীকে বিভিন্ন মেয়াদে সশ্রম কারাদণ্ড দেওয়া হয়েছে । [SEP]
segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
is_random_next: False
mlm_positions: 2 5
mlm_labels: ছাড়া নয়জন


tokens: [CLS] সেদিন রাতেই জানা গেল চিকিৎসাধীন 

tokens: [CLS] দুই বিস্ফোরণে র্যাবের গোয়েন্দাপ্রধান দুজন পুলিশ কর্মকর্তাসহ সাতজন [MASK] হন [MASK] [SEP]
segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0
is_random_next: False
mlm_positions: 9 11
mlm_labels: নিহত ।


tokens: [CLS] আর র্যাবের সদস্যরা পুরো [MASK] কক্ষগুলো ঘুরে পেলেন [MASK] বেড । [SEP]
segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0
is_random_next: False
mlm_positions: 5 9
mlm_labels: ভবনের 1111111111


tokens: [CLS] র্যাবের নির্বাহী হাকিম [MASK] আলম প্রথম [MASK] বলেন ইলিশ এ দেশের জাতীয় সম্পদ । [SEP]
segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
is_random_next: False
mlm_positions: 4 7
mlm_labels: সারওয়ার আলোকে


tokens: [CLS] [MASK] [MASK] মামলায় মৃত্যুদণ্ডে দণ্ডিত হয়েছেন একজন সাবেক লেফটেন্যান্ট কর্নেলসহ র্যাবের 1111111111 জন [MASK] । [SEP]
segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
is_random_next: False
mlm_positions: 1 2 14
mlm_labels: চাঞ্চল্যকর এ সদস্য


tokens: [CLS] র্যাবের গণমাধ্যম শাখা থেকে [MASK] [MASK] জানানো হয়েছে । [SEP]
segment_ids: 0 0 0 0 0 0 0 0 0 0 0
is_random_next: False

In [68]:
text = 'এ ছাড়া শিক্ষাপ্রতিষ্ঠানেও চলবে প্রচারণা।'
encoded = tokenizer(text)
# encoded["input_ids"].shape

tokenizer.decode(encoded["input_ids"])

'[CLS] এ ছাড়া শিক্ষাপ্রতিষ্ঠান চলবে প্রচারণা । [SEP]'